In [1]:
%%bash

pip install --upgrade pip
pip install farm-haystack[colab,faiss]==1.17.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 10.4 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.6/64.6 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.3/112.3 kB 6.6 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of sentence-transformers to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of sentence-transformers to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 7.4 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
INFO: pip is looking at multiple versions of pydantic to determine which version is compatible with other requir

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython-sql 0.5.0 requires sqlalchemy>=2.0, but you have sqlalchemy 1.4.52 which is incompatible.
pydantic-core 2.18.4 requires typing-extensions!=4.7.0,>=4.6.0, but you have typing-extensions 4.5.0 which is incompatible.
tensorflow 2.15.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 3.20.2 which is incompatible.
tensorflow-metadata 1.15.0 requires protobuf<4.21,>=3.20.3; python_version < "3.11", but you have protobuf 3.20.2 which is incompatible.
torchaudio 2.3.0+cu121 requires torch==2.3.0, but you have torch 2.1.2 which is incompatible.
torchtext 0.18.0 requires torch>=2.3.0, but you have torch 2.1.2 which is incompatible.


example


In [6]:
from haystack.nodes import RAGenerator, DensePassageRetriever
from haystack.document_stores import FAISSDocumentStore
from haystack.document_stores import InMemoryDocumentStore
from haystack.nodes import BM25Retriever
from haystack.nodes import FARMReader
from haystack.pipelines import ExtractiveQAPipeline
from pydantic import BaseModel

reader = FARMReader(model_name_or_path="deepset/xlm-roberta-large-squad2")


class Qna(BaseModel):
    context: str
    question: str


def question(qna: Qna):
    document_store = InMemoryDocumentStore(use_bm25=True)

    # token = jwt.decode(request.headers["authorization"].replace("Bearer ", ""), options={"verify_signature": False})
    # data = await request.app.state.database.content.find({"userId": token["profileId"]}).to_list(length=1000)
    # dict_list = [{"content": item["content"]} for item in data if item["content"]]

    document_store.write_documents([{"content": qna.context}])

    retriever = BM25Retriever(document_store=document_store)
    pipeline = ExtractiveQAPipeline(reader, retriever)
    response = pipeline.run(query=qna.question)
    answer_object = response["answers"][0]

    print("-----------------")
    print(answer_object.answer)
    print("-----------------")

    return {"answer": answer_object.answer, "score": answer_object.score + 0.2, "context": answer_object.context}



question(Qna(context="프랑스의 수도는 파리야", question="프랑스의 수도는 어디야?"))




Updating BM25 representation...:   0%|          | 0/1 [00:00<?, ? docs/s]

Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]

-----------------
파리
-----------------


{'answer': '파리', 'score': 1.1737364053726196, 'context': '프랑스의 수도는 파리야'}